Credit Card Fraud Detection::
Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

Description about dataset::
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning.

Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

WORKFLOW :


1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables.

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 
  10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92% 

10.Evaluation Step 

11. Prediction




In [6]:

from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/MyDrive/Colab Notebooks/creditcard.csv"

import pandas as pd
import numpy as np

df = pd.read_csv(path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
data = df.iloc[:, :-1]
targets = df.iloc[:, -1]

print(data.shape)
print(targets.shape)

(284807, 30)
(284807,)


In [9]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.3)
train_data, validation_data, train_targets, validation_targets = train_test_split(train_data, train_targets, test_size=.2)

In [10]:
## Standardizing Data

mean = np.mean(train_data)
std = np.std(train_data)

train_data -= mean
train_data /= std

validation_data -= mean
validation_data /= std

test_data -= mean
test_data /= std

In [11]:
from keras import models, layers

model = models.Sequential()

model.add(layers.Dense(10, input_shape=(train_data.shape[1],), activation='relu'))

model.add(layers.Dense(8, activation='relu'))

model.add(layers.Dense(6, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

In [12]:
from sklearn.metrics import classification_report

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_targets, epochs=100, batch_size=64)


Epoch 1/100
2493/2493 [==============================] - 4s 1ms/step - loss: 0.1099 - accuracy: 0.9685
Epoch 2/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0070 - accuracy: 0.9993
Epoch 3/100
2493/2493 [==============================] - 4s 1ms/step - loss: 0.0042 - accuracy: 0.9994
Epoch 4/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0049 - accuracy: 0.9995
Epoch 5/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0048 - accuracy: 0.9994
Epoch 6/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0046 - accuracy: 0.9995
Epoch 7/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0051 - accuracy: 0.9994
Epoch 8/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0046 - accuracy: 0.9994
Epoch 9/100
2493/2493 [==============================] - 3s 1ms/step - loss: 0.0038 - accuracy: 0.9995
Epoch 10/100
2493/2493 [==============================] - 3s 1ms/step - l

In [13]:
val_predictions = model.predict(validation_data)
print(val_predictions)

[[1.5407149e-05]
 [1.9770006e-05]
 [2.5536451e-06]
 ...
 [3.5165220e-05]
 [1.3145873e-05]
 [6.2934223e-06]]


In [14]:
preds = np.around(val_predictions)
preds

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [15]:
print(classification_report(validation_targets, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39801
           1       0.76      0.74      0.75        72

    accuracy                           1.00     39873
   macro avg       0.88      0.87      0.87     39873
weighted avg       1.00      1.00      1.00     39873



In [16]:
loss, accuracy = model.evaluate(test_data, test_targets)
print(loss)
print(accuracy)

2671/2671 [==============================] - 3s 964us/step - loss: 0.0043 - accuracy: 0.9995
0.0042861755937337875
0.9994733333587646


In [17]:
test_preds = model.predict(test_data)
test_preds.shape

(85443, 1)

In [18]:
test_preds = test_preds.reshape(85443)
test_preds = np.round(test_preds)
print(classification_report(test_targets, test_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85297
           1       0.89      0.79      0.84       146

    accuracy                           1.00     85443
   macro avg       0.94      0.90      0.92     85443
weighted avg       1.00      1.00      1.00     85443



In [19]:
output_df = pd.DataFrame({'Actual': test_targets, 'Prediction': test_preds})
print(output_df.head())
print(output_df.shape)

        Actual  Prediction
188844       0         0.0
148210       0         0.0
177735       0         0.0
13617        0         1.0
108549       0         0.0
(85443, 2)


In [20]:
## How many correct predictions?

filt = (output_df.loc[:, 'Actual'] == output_df.loc[:, 'Prediction'])

output_df[filt].shape

(85398, 2)

In [21]:
print(f'Accuracy is {(output_df[filt].shape[0]/output_df.shape[0])*100}%')

Accuracy is 99.94733330992591%
